In [ ]:
import sys
from dotenv import load_dotenv
import os

load_dotenv()

sys.path.append(os.getenv("ROOT"))
os.getenv("ROOT")

In [ ]:
print(sys.path)

In [ ]:
import pandas as pd
import polars as pl
import numpy as np
from datetime import date
from silverfund.datasets import CRSPDaily, Universe
import seaborn as sns
import matplotlib.pyplot as pltc
from matplotlib import pyplot as plt

In [ ]:
import statsmodels.formula.api as smf
from finance_byu.regtables import Regtable

In [ ]:
# df = (CRSPDaily(start_date=date(2010, 1, 1),
#                 end_date=date(2015, 12, 31))
#                 .load_all()
#                 .select(["permno", "ticker", "date", "shrout", "prc", "ret"])
# )

# df

# df = (Universe(start_date=date(2010, 1, 1),
#                 end_date=date(2015, 12, 31))
#                 .load_all()
#                 .select(["permno", "ticker", "date", "shrout", "prc", "ret"])
# )

# df

In [ ]:
path = "/home/bwaits/fsl_groups/grp_quant/data/daily_merged/"
files = os.listdir(path)
dfs = []
for file in files:
    # if file.endswith('.parquet'):
    for year in range(2010, 2024):
        if file.endswith(f"mm_{year}.parquet"):
            df = pl.read_parquet(f"{path}/{file}").to_pandas()

            dfs.append(df)


df = pd.concat(dfs)
df2 = df.copy()

In [ ]:
df["DataDate"] = pd.to_datetime(df["DataDate"], format="%Y%m%d")
df = df[
    ["DataDate", "barrid", "ret_barra", "vol_crsp", "price_barra", "mktcap_barra"]
]  #'lag_ret', 'signal'
df["logret"] = np.log1p(df["ret_barra"])
df["rev"] = df.groupby("barrid")["logret"].rolling(22, 22).sum().reset_index(drop=True)
df["vol"] = df.groupby("barrid")["vol_crsp"].rolling(22, 22).sum().reset_index(drop=True)
df["rev"] = df.groupby("barrid")["rev"].shift(1)
df["prclag"] = df.groupby("barrid")["price_barra"].shift(1)
df["melag"] = df.groupby("barrid")["mktcap_barra"].shift(1)
df = df.query("rev == rev and prclag >= 5").reset_index(drop=True)
df["bins"] = df.groupby("DataDate")["rev"].transform(pd.qcut, 10, labels=False)
df

In [ ]:
reg_df = df[["rev", "vol"]]
reg_df
# reg = smf.ols('rev ~ 1 + vol', data=reg_df).fit()
# #reg.summary()
# Regtable(reg,sig='coeff').render()

In [ ]:
def OLS(X, y):

    X = np.hstack([np.ones((X.shape[0], 1)), X])
    n, m = X.shape
    beta = np.linalg.pinv(X.T @ X) @ (X.T @ y)
    print(X)

    # tstats from chat
    # Calculate residuals
    y_pred = X @ beta
    residuals = y - y_pred
    # Residual variance (sigma^2)
    residual_var = (residuals.T @ residuals) / (n - m)
    # Variance-covariance matrix of the coefficients: Var(beta) = sigma^2 * (X^T X)^(-1)
    cov_matrix = residual_var * np.linalg.inv(X.T @ X)
    # Standard errors of the coefficients (sqrt of diagonal elements of cov_matrix)
    stderr = np.sqrt(np.diag(cov_matrix))
    # T-statistics: t = beta / stderr
    tstats = beta / stderr
    return beta, tstats, stderr


X = np.array(df["vol"]).reshape(-1, 1)
y = np.array(df["rev"])
coef, t_stats, se = OLS(X, y)

print("Coefficients:", coef)
print("Standard Errors:", se)
print("T-Statistics:", t_stats)

In [ ]:
df.groupby(["DataDate", "bins"])["logret"].count()

In [ ]:
ew = df.groupby(["DataDate", "bins"])["logret"].mean().unstack(level="bins") * 100

ew["spread"] = ew[0] - ew[9]
ew

In [ ]:
ew.cumsum().plot(figsize=(8, 5))
plt.title("Equal Weight Full Sample PnL")
plt.ylabel("Cumulative Log Returns")
plt.xlabel("Date")
plt.show()

In [ ]:
mcapsum = df.groupby(["DataDate", "bins"])["melag"].sum()
df["rme"] = df["logret"] * df["melag"]
vw = df.groupby(["DataDate", "bins"])["rme"].sum() / mcapsum
vw = vw.unstack(level="bins") * 100
vw["spread"] = vw[0] - vw[9]
vw

In [ ]:
vw.cumsum().plot(figsize=(8, 5))
plt.title("Value Weight Full Sample PnL")
plt.ylabel("Cumulative Log Returns")
plt.xlabel("Date")
plt.show()

In [ ]:
# df['logret'] = np.log1p(df['ret_barra'])
# df['rev'] = df.groupby('barraid')['logret'].rolling(22,22).sum().reset_index(drop=True)
# df['rev'] = df.groupby('barraid')['rev'].shift(2)
# df.head(15)

## Seth's Method

In [ ]:
df = df2.copy()

In [ ]:
df

In [ ]:
# df = pl.read_parquet("/home/bwaits/fsl_groups/grp_quant/data/daily_merged/mm_2024.parquet")
# df = df.to_pandas()
# #df.index = df['DataDate']
# #df = df[['DataData', 'barrid', 'ret_barra']]
# df

In [ ]:
# df = pd.concat(dfs)
df["DataDate"] = pd.to_datetime(df["DataDate"], format="%Y%m%d")
# df.index = pd.to_datetime(df.index, format='%Y%m%d')
df["lag_ret"] = np.log1p(df.groupby("barrid")["ret_barra"].shift(1))
mean = df.groupby("DataDate")["lag_ret"].mean()
std = df.groupby("DataDate")["lag_ret"].std()

df["sig_mean"] = df["DataDate"].map(mean)
df["sig_std"] = df["DataDate"].map(std)
# df['sig_mean'] = df.index.map(mean)
# df['sig_std'] = df.index.map(std)

# Mean reversion signal
df["signal"] = -1 * (df["lag_ret"] - df["sig_mean"]) / df["sig_std"]
df["signal"] = df["signal"]
# Roughly mean zero and std one.. good!
# df = df[['DataDate', 'barrid', 'lag_ret', 'signal', 'vol_crsp']]
df = df[
    ["DataDate", "barrid", "lag_ret", "vol_crsp", "signal", "price_barra", "mktcap_barra"]
]  #'lag_ret', 'signal'
df

In [ ]:
# df['rev'] = df.groupby('barrid')['logret'].rolling(22,22).sum().reset_index(drop=True)
# df['rev'] = df.groupby('barrid')['rev'].shift(2)

df["prclag"] = df.groupby("barrid")["price_barra"].shift()
df["melag"] = df.groupby("barrid")["mktcap_barra"].shift()
df = df.query("signal == signal and prclag >= 5").reset_index(drop=True)
df["bins"] = df.groupby("DataDate")["signal"].transform(pd.qcut, 10, labels=False)
df

In [ ]:
ew = df.groupby(["DataDate", "bins"])["lag_ret"].mean().unstack(level="bins") * 100

ew["spread"] = ew[0] - ew[9]
ew

In [ ]:
ew.cumsum().plot(figsize=(8, 5))
plt.title("Equal Weight Full Sample PnL")
plt.ylabel("Cumulative Log Returns")
plt.xlabel("Date")
plt.show()

In [ ]:
mcapsum = df.groupby(["DataDate", "bins"])["melag"].sum()
df["rme"] = df["lag_ret"] * df["melag"]
vw = df.groupby(["DataDate", "bins"])["rme"].sum() / mcapsum
vw = vw.unstack(level="bins") * 100
vw["spread"] = vw[0] - vw[9]
vw

In [ ]:
vw.cumsum().plot(figsize=(8, 5))
plt.title("Value Weight Full Sample PnL")
plt.ylabel("Cumulative Log Returns")
plt.xlabel("Date")
plt.show()

In [ ]:
# Roughly mean zero and std one.. good!
# df = df[['DataDate', 'barrid', 'lag_ret', 'signal', 'vol_crsp']]

# %%
# alpha = df.pivot(index='DataDate', columns='barrid', values='signal')
# # alpha = df.pivot(index=df.index, columns='!Barrid', values='signal')
# alpha.to_parquet('/home/bwaits/Research/sf-trading-system/reversal_research/alphas.parquet')
# alpha

In [ ]:
# import pandas as pd
# from silverfund.backtester import MVBacktester
# from silverfund.components.optimizers.constraints import *

In [ ]:
# # %%
# alphas_df = pd.read_parquet('/home/bwaits/Research/sf-trading-system/reversal_research/alphas.parquet')
# # %%
# mv_backtester = MVBacktester(
#     alpha = alphas_df.copy(),
#     constraints = [FullInvestmentConstraint()]
# )
# # %%
# port = mv_backtester.get_optimal_portfolio_history(n_cpus=16)
# # %%
# port.to_parquet('/home/bwaits/Research/sf-trading-system/reversal_research/portfolios.parquet')

In [ ]:
df